In [9]:
import pandas as pd
import json
from brokerage.oanda.oanda import Oanda

In [10]:
db_file = "oan_hourly_ohlcv.xlsx"
with open("config/portfolio_config.json", "r") as f:
    portfolio_config = json.load(f)

brokerage_used = portfolio_config["brokerage"]
brokerage_config_path = portfolio_config["brokerage_config"][brokerage_used]

with open("config/{}".format(brokerage_config_path), "r") as f:
    brokerage_config = json.load(f)

with open("config/auth_config.json", "r") as f:
    auth_config = json.load(f)

In [25]:
database_df = pd.read_excel("./Data/{}".format(db_file)).set_index("date")
database_df = database_df.loc[~database_df.index.duplicated(keep="first")] 
brokerage = Oanda(brokerage_config=brokerage_config, auth_config=auth_config)

#by default, main does not train the classifier
run_live_classifier = False
#if running test , only loads from disk

db_instruments = brokerage_config["fx"] +  brokerage_config["indices"] + brokerage_config["commodities"] + brokerage_config["metals"] + brokerage_config["bonds"]
use_disk = portfolio_config["use_disk"]
poll_df = pd.DataFrame()
for db_inst in db_instruments:
    tries = 0
    again = True
    while again:
        try:
            df = brokerage.get_trade_client().get_hourly_ohlcv(instrument=db_inst, count=50, granularity="H1")
            df.set_index("date", inplace=True)
            #print(db_inst, "\n", df)
            cols = list(map(lambda x: "{} {}".format(db_inst, x), df.columns)) 
            df.columns = cols                
            if len(poll_df) == 0:
                poll_df[cols] = df
            else:
                poll_df = poll_df.combine_first(df)
            again = False
        except Exception as err:
            print(err)
            tries += 1
            if tries >=5:
                again=False
                print("Check TCP Socket Connection, rerun application")
                exit()

In [26]:
database_df

,USD_MXN open,USD_MXN high,USD_MXN low,USD_MXN close,USD_MXN volume,GBP_USD open,GBP_USD high,GBP_USD low,GBP_USD close,GBP_USD volume,...,UK10YB_GBP open,UK10YB_GBP high,UK10YB_GBP low,UK10YB_GBP close,UK10YB_GBP volume,USB30Y_USD open,USB30Y_USD high,USB30Y_USD low,USB30Y_USD close,USB30Y_USD volume
date,,,,,,,,,,,,,,,,,,,,,
2021-06-04 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-04 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-04 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-07 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-07 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-12 16:00:00,17.31471,17.33558,17.30260,17.30465,6416.0,1.24890,1.24999,1.24884,1.24945,3893.0,...,95.526,95.606,95.486,95.526,445.0,126.436,126.530,126.374,126.468,115.0
2023-06-12 17:00:00,17.30440,17.30740,17.28323,17.28402,7256.0,1.24946,1.25040,1.24926,1.25037,3581.0,...,NaN,NaN,NaN,NaN,NaN,126.452,126.592,126.250,126.483,173.0
2023-06-12 18:00:00,17.28407,17.29720,17.28085,17.29224,7615.0,1.25036,1.25078,1.24987,1.25050,2697.0,...,NaN,NaN,NaN,NaN,NaN,126.468,126.670,126.468,126.654,136.0


In [27]:
poll_df = poll_df.tail(50)

In [28]:
poll_df

,AU200_AUD close,AU200_AUD high,AU200_AUD low,AU200_AUD open,AU200_AUD volume,AUD_USD close,AUD_USD high,AUD_USD low,AUD_USD open,AUD_USD volume,...,XPD_USD close,XPD_USD high,XPD_USD low,XPD_USD open,XPD_USD volume,XPT_USD close,XPT_USD high,XPT_USD low,XPT_USD open,XPT_USD volume
date,,,,,,,,,,,,,,,,,,,,,
2023-06-09 18:00:00,7097.3,7103.8,7091.8,7103.8,307.0,NaN,NaN,NaN,NaN,NaN,...,1310.690,1313.183,1308.694,1309.692,185.0,1011.102,1013.594,1010.752,1013.594,607.0
2023-06-09 19:00:00,7095.8,7100.3,7092.8,7097.8,579.0,0.67396,0.67407,0.67374,0.67402,1374.0,...,1312.188,1316.675,1310.939,1310.939,471.0,1011.154,1012.150,1010.752,1011.152,821.0
2023-06-09 20:00:00,7091.8,7101.8,7091.8,7096.3,92.0,0.67437,0.67440,0.67394,0.67394,687.0,...,1313.684,1316.427,1310.940,1311.938,583.0,1010.556,1011.803,1010.307,1011.104,238.0
2023-06-11 21:00:00,NaN,NaN,NaN,NaN,NaN,0.67398,0.67415,0.67379,0.67379,398.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-11 22:00:00,NaN,NaN,NaN,NaN,NaN,0.67409,0.67441,0.67391,0.67398,738.0,...,1305.026,1314.998,1305.026,1313.252,106.0,1009.014,1011.257,1008.864,1010.259,286.0
2023-06-11 23:00:00,NaN,NaN,NaN,NaN,NaN,0.67448,0.67462,0.67405,0.67410,675.0,...,1306.772,1307.768,1303.529,1303.778,253.0,1009.366,1010.113,1008.565,1008.914,243.0
2023-06-12 00:00:00,NaN,NaN,NaN,NaN,NaN,0.67442,0.67484,0.67396,0.67448,1486.0,...,1309.516,1311.260,1306.522,1307.022,525.0,1008.270,1009.865,1008.270,1009.416,465.0
2023-06-12 01:00:00,NaN,NaN,NaN,NaN,NaN,0.67378,0.67458,0.67366,0.67441,2205.0,...,1306.526,1310.014,1304.280,1309.766,442.0,1005.080,1008.370,1004.830,1008.220,927.0
2023-06-12 02:00:00,NaN,NaN,NaN,NaN,NaN,0.67366,0.67406,0.67314,0.67377,1724.0,...,1306.526,1307.274,1301.290,1306.774,265.0,1004.284,1005.628,1003.184,1005.030,423.0


In [29]:
database_df = database_df.loc[:poll_df.index[0]][:-1]
#database_df = database_df.append(poll_df)
joined = pd.concat([database_df, poll_df],axis=0)
print('01: Appened df')
#print(database_df)

01: Appened df


In [37]:
joined['USD_MXN open'].tail(50).values

array([     nan, 17.27046, 17.27289, 17.27925, 17.30015, 17.2901 ,
       17.2853 , 17.2952 , 17.30053, 17.29962, 17.29708, 17.294  ,
       17.29272, 17.2866 , 17.31104, 17.28825, 17.2851 , 17.28605,
       17.2869 , 17.2614 , 17.2918 , 17.32436, 17.31471, 17.3044 ,
       17.28407, 17.29228, 17.27418, 17.2847 , 17.28498, 17.2817 ,
       17.28715, 17.29244, 17.28575, 17.28464, 17.28202, 17.28358,
       17.29236, 17.29995, 17.28564, 17.31172, 17.30898, 17.31646,
       17.2951 , 17.25924, 17.27089, 17.2423 , 17.21943, 17.22085,
       17.21626, 17.2128 ])

In [36]:
database_df['USD_MXN open'].tail(100).values

array([17.49148, 17.47297, 17.46938, 17.4716 , 17.463  , 17.46265,
       17.46814, 17.46405, 17.46369, 17.4611 , 17.46635, 17.46154,
       17.46065, 17.45411, 17.45543, 17.44865, 17.4513 , 17.45044,
       17.4371 , 17.45138, 17.4431 , 17.44825, 17.4528 , 17.46968,
       17.459  , 17.44258, 17.41732, 17.40705, 17.40211, 17.40581,
       17.3939 , 17.3954 , 17.38695, 17.38414, 17.3795 , 17.37652,
       17.37686, 17.38055, 17.3777 , 17.37772, 17.38436, 17.3798 ,
       17.3698 , 17.36314, 17.353  , 17.32062, 17.369  , 17.3402 ,
       17.31546, 17.37244, 17.34567, 17.34435, 17.36035, 17.35086,
       17.35758, 17.364  , 17.36385, 17.36718, 17.35835, 17.35451,
       17.35252, 17.35285, 17.3555 , 17.3601 , 17.35165, 17.3493 ,
       17.346  , 17.34385, 17.3436 , 17.34542, 17.33815, 17.36114,
       17.39224, 17.42826, 17.4142 , 17.39233, 17.40338, 17.39514,
       17.39212, 17.38145, 17.3738 , 17.38447, 17.38563, 17.3874 ,
       17.3829 , 17.38551, 17.3819 , 17.385  , 17.37413, 17.35

In [44]:
joined.fillna(method="ffill", inplace=True) #fill missing data by first forward filling data, such that [] [] [] a b c [] [] [] becomes [] [] [] a b c c c c
joined.fillna(method="bfill", inplace=True) 

In [46]:
joined.isna().any()[lambda x: x]

Series([], dtype: bool)